This version aims to provide a more optimised implementation of the Apriori algorithm.

If a set is found to be infrequent, there is no need to consider its supersets due to the antimonotone property.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/processed/Oct2019Purchases.csv')
df = df['product_id']
df #all transactions only 

0                           [8500083, 8500084]
1                                    [5100443]
2         [9300037, 9300037, 9300037, 2100099]
3                                   [17301479]
4                                    [2501816]
                          ...                 
347113                              [22700129]
347114                    [12708874, 12704161]
347115                               [1005136]
347116                              [12702883]
347117                               [1005144]
Name: product_id, Length: 347118, dtype: object

In [3]:
transactions = []
for i in df:
    t = i[1:-1]
    ls = t.split(', ')
    transactions.append(set(ls))


In [4]:
transactions

[{'8500083', '8500084'},
 {'5100443'},
 {'2100099', '9300037'},
 {'17301479'},
 {'2501816'},
 {'10700971'},
 {'12718429'},
 {'1801805'},
 {'13300559'},
 {'1003304', '1004836'},
 {'1003304'},
 {'26400256', '26400485'},
 {'3500303'},
 {'1004858'},
 {'9100708'},
 {'23300493'},
 {'1307240'},
 {'1004210'},
 {'22700067'},
 {'7101297'},
 {'1004767', '1004838'},
 {'12700979'},
 {'3600661'},
 {'1004856'},
 {'1004833'},
 {'1004505'},
 {'12703015'},
 {'2501202'},
 {'26300095', '26300188', '26300354'},
 {'34700074'},
 {'5100816'},
 {'1004875'},
 {'5100572'},
 {'1004838', '1004839', '5301587'},
 {'26402783'},
 {'1004741'},
 {'25500275'},
 {'1004857'},
 {'2701646', '31500231'},
 {'26403341'},
 {'1004838'},
 {'29501743'},
 {'1002544'},
 {'1500258'},
 {'1004781', '1004870'},
 {'2201033'},
 {'1005252'},
 {'1004858', '1801739'},
 {'11000126'},
 {'1004249'},
 {'2800203'},
 {'26500121'},
 {'26402587'},
 {'22600031'},
 {'1002628', '8700375'},
 {'1003316'},
 {'4400225'},
 {'1004777'},
 {'1004856'},
 {'37009

In [5]:
from itertools import combinations

def find_frequent_itemsets(transactions, minsup):
    itemsets = {}
    transaction_list = list(transactions)
    
    # Step 1: Find all frequent itemsets
    k = 1
    while True:
        # Generate candidate k-itemsets
        ck = {}
        for transaction in transaction_list:
            for itemset in combinations(transaction, k): #all possible subsets of the transaction, of length k
                if itemset in ck:
                    ck[itemset] += 1
                else:
                    ck[itemset] = 1
        
        # Prune infrequent k-itemsets. If a set is found to be infrequent, there is no need to consider supersets due to antimonotone property.
        lk = {itemset: support for itemset, support in ck.items() if support >= minsup}
        if not lk:
            break #no frequent itemsets for this particular value of k, no need to keep generating more itemsets.
        itemsets[k] = lk
        print(k)
        k += 1
    
    return itemsets

def generate_association_rules(itemsets, minconf):
    rules = []
    for size, itemset_list in itemsets.items():
        if size == 1:
            continue
        
        for itemset in itemset_list:
            for antecedent_size in range(1, size):
                for antecedent in combinations(itemset, antecedent_size):
                    consequent = tuple(set(itemset) - set(antecedent))
                    
                    if len(consequent) > 0:
                        confidence = itemsets[size][itemset] / itemsets[antecedent_size][antecedent]
                        if confidence >= minconf:
                            rules.append((antecedent, consequent, confidence))
    
    return rules

# Example usage
# transactions = [
    
#     {"milk", "bread", "nuts", "apple"},
#     {"milk", "bread", "nuts"},
#     {"milk", "nuts"},
#     {"milk", "bread", "apple"},
#     {"milk", "bread"},
#     {"milk", "bread", "nuts"},
#     {"nuts", "bread"}
# ]

# milk -> nuts 
# milk + nuts support / milk support

minsup = 2
minconf = 0.6

frequent_itemsets = find_frequent_itemsets(transactions, minsup)
association_rules = generate_association_rules(frequent_itemsets, minconf)

# Print frequent itemsets and association rules

with open('results/Oct19purchases.txt', 'w') as f:
    f.write("Frequent Itemsets:\n")
    for size, itemset_list in frequent_itemsets.items():
        for itemset in itemset_list:
            f.write(f"{itemset}: Support = {frequent_itemsets[size][itemset]}")
            f.write('\n')

    f.write("\nAssociation Rules:")
    for rule in association_rules:
        antecedent, consequent, confidence = rule
        f.write(f"{antecedent} -> {consequent}: Confidence = {confidence}")
        f.write('\n')


1
2
3
4
5


KeyboardInterrupt: 

In [18]:
from collections import defaultdict

def load_data():
    # Sample transaction data
    data = [
    {"milk", "bread", "nuts", "apple"},
    {"milk", "bread", "nuts"},
    {"milk", "nuts"},
    {"milk", "bread", "apple"},
    {"milk", "bread"},
    {"milk", "bread", "nuts"},
    {"nuts", "bread"}
    ]
    return data

def generate_c1(data):
    """
    Generate all candidate sets with size 1
    """
    c1 = set()
    for transaction in data:
        for item in transaction:
            c1.add(frozenset([item])) #frozensetis immutable so that it can be used as key in dictionary later 
    return c1

def generate_l1(data, c1, min_support):
    """
    Generate all frequent itemsets with size 1
    """
    item_counts = defaultdict(int) #defaultdict automatically creates keys
    for transaction in data:
        for itemset in c1:
            if itemset.issubset(transaction):
                item_counts[itemset] += 1

    l1 = set()
    for itemset, count in item_counts.items():
        if count >= min_support:
            l1.add(itemset)
    
    return l1

def generate_candidates(lk, k):
    candidates = set()
    for item1 in lk:
        for item2 in lk:
            if list(item1)[:k] == list(item2)[:k]:
                candidate = item1.union(item2)
                # Check if all subsets are frequent
                valid = True
                for subset in candidate:
                    if candidate - set([subset]) not in lk:
                        valid = False
                        break
                if valid:
                    candidates.add(candidate)
    return candidates

def calculate_confidence(itemset, antecedent, support_counts):
    if antecedent not in support_counts:
        return 0
    return support_counts[itemset]/support_counts[antecedent]

def apriori(data, min_support, min_confidence):
    c1 = generate_c1(data)
    l1 = generate_l1(data, c1, min_support)
    frequent_itemsets = [l1]
    support_counts = defaultdict(int)
    k = 1
    while len(frequent_itemsets[k-1]) > 0:
        ck = generate_candidates(frequent_itemsets[k-1], k)
        item_counts = defaultdict(int)
        for transaction in data:
            for candidate in ck:
                if candidate.issubset(transaction):
                    item_counts[candidate] += 1
        lk = set()
        for itemset, count in item_counts.items():
            if count >= min_support:
                confidence = calculate_confidence(itemset, itemset.difference([list(itemset)[0]]), support_counts)
                if confidence >= min_confidence:
                    lk.add(itemset)
                support_counts[itemset] = count
        frequent_itemsets.append(lk)
        k += 1
    return frequent_itemsets

def print_frequent_itemsets(frequent_itemsets):
    for k, itemsets in enumerate(frequent_itemsets, start=1):
        print(f"Frequent {k}-itemsets:")
        for itemset in itemsets:
            print(f"{list(itemset)}")

data = load_data()
min_support = 2
min_confidence = 0.5
frequent_itemsets = apriori(data, min_support, min_confidence)
print_frequent_itemsets(frequent_itemsets)


Frequent 1-itemsets:
['apple']
['nuts']
['bread']
['milk']
Frequent 2-itemsets:
